In [1]:
# 1. Mount Google Drive and Install Dependencies
from google.colab import drive
drive.mount('/content/drive')
!pip install mlflow tensorflow wandb

# 2. Import Libraries
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
import mlflow
import mlflow.tensorflow
import wandb
wandb.login()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


wandb: Currently logged in as: jd-lopez160 (jd-lopez160-pontificia-universidad-javeriana). Use `wandb login --relogin` to force relogin


True

# Train From Scratch


In [2]:
import os
import shutil

# Rutas de origen en Google Drive
dir_covid = "/content/drive/MyDrive/Proyecto 1/Imagenes Proyecto 1/Procesadas/Covid"
dir_pneumonia = "/content/drive/MyDrive/Proyecto 1/Imagenes Proyecto 1/Procesadas/Pneumonia"
dir_normal = "/content/drive/MyDrive/Proyecto 1/Imagenes Proyecto 1/Procesadas/Normal"

# Directorios de destino en /content/
dest_covid = "/content/Covid"
dest_pneumonia = "/content/Pneumonia"
dest_normal = "/content/Normal"

# Crear los directorios de destino si no existen
os.makedirs(dest_covid, exist_ok=True)
os.makedirs(dest_pneumonia, exist_ok=True)
os.makedirs(dest_normal, exist_ok=True)

# Copiar los archivos
shutil.copytree(dir_covid, dest_covid, dirs_exist_ok=True)
shutil.copytree(dir_pneumonia, dest_pneumonia, dirs_exist_ok=True)
shutil.copytree(dir_normal, dest_normal, dirs_exist_ok=True)

print("Archivos copiados a /content/")

Archivos copiados a /content/


In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Configuración para la precisión mixta para aprovechar mejor la GPU A100

# Rutas locales de los datos
dir_covid = "/content/Covid"
dir_pneumonia = "/content/Pneumonia"
dir_normal = "/content/Normal"

def build_model():
    model = models.Sequential([
        layers.Input(shape=(299, 299, 3)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(512, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(512, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dense(4096, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])
    return model

# Crear el generador de imágenes con el tamaño de lote adecuado para la A100
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Generadores para entrenamiento y validación usando las rutas locales
train_generator = train_datagen.flow_from_directory(
    '/content/DATASET/',  # Ruta base local donde se encuentran los subdirectorios
    target_size=(299, 299),
    batch_size=128,  # Tamaño de lote aumentado
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    '/content/DATASET/',  # Ruta base local donde se encuentran los subdirectorios
    target_size=(299, 299),
    batch_size=128,  # Tamaño de lote aumentado
    class_mode='categorical',
    subset='validation')

# Construir y compilar el modelo
model = build_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



Found 16808 images belonging to 3 classes.
Found 4201 images belonging to 3 classes.


In [8]:
# Entrenar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 128,  # Ajustado al tamaño del lote
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 128,  # Ajustado al tamaño del lote
    epochs=10  # Ajusta el número de épocas según sea necesario
)



Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


131/131 ━━━━━━━━━━━━━━━━━━━━ 112s 549ms/step - accuracy: 0.5428 - loss: 1.7655 - val_accuracy: 0.4856 - val_loss: 1.0913
Epoch 2/10
  1/131 ━━━━━━━━━━━━━━━━━━━━ 17s 138ms/step - accuracy: 0.7578 - loss: 0.5726

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.7578 - loss: 0.5726 - val_accuracy: 0.4476 - val_loss: 1.0946
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 53s 389ms/step - accuracy: 0.7805 - loss: 0.5156 - val_accuracy: 0.6592 - val_loss: 1.3406
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step - accuracy: 0.8594 - loss: 0.3683 - val_accuracy: 0.6762 - val_loss: 0.9839
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 53s 389ms/step - accuracy: 0.8395 - loss: 0.3669 - val_accuracy: 0.6719 - val_loss: 1.0826
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 897us/step - accuracy: 0.8984 - loss: 0.2664 - val_accuracy: 0.6762 - val_loss: 1.0287
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 53s 389ms/step - accuracy: 0.8669 - loss: 0.3120 - val_accuracy: 0.6594 - val_loss: 2.5189
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - accuracy: 0.8594 - loss: 0.3511 - val_accuracy: 0.6000 - val_loss: 2.6960
Epoch 9/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 53s 389ms/step - accuracy: 0.8898 - loss: 0.2648 - val_accu

In [ ]:
# Si deseas guardar el modelo entrenado:
model.save('/content/my_model.h5')


## Big Training Run With Experiments

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import os
import wandb

# Ruta base de datos
base_dir = '/content/DATASET/'

# Función para preparar los datasets
def get_datasets(scenario):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    if scenario == 'original':
        # Usar los datos directamente del directorio
        train_generator = datagen.flow_from_directory(
            base_dir,
            target_size=(299, 299),
            batch_size=128,
            class_mode='categorical',
            subset='training'
        )
        val_generator = datagen.flow_from_directory(
            base_dir,
            target_size=(299, 299),
            batch_size=128,
            class_mode='categorical',
            subset='validation'
        )
    else:
        # Implementación simplificada para undersampling y oversampling
        all_generator = datagen.flow_from_directory(
            base_dir,
            target_size=(299, 299),
            batch_size=128,
            class_mode='categorical',
            subset=None,
            shuffle=False
        )
        x, y = all_generator.next()
        if scenario == 'undersampling':
            sampler = RandomUnderSampler(random_state=42)
        else:
            sampler = RandomOverSampler(random_state=42)
        x_res, y_res = sampler.fit_resample(np.reshape(x, (x.shape[0], -1)), np.argmax(y, axis=1))
        x_res = np.reshape(x_res, (x_res.shape[0], 299, 299, 3))

        # Rehacer los generadores
        train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
        train_generator = train_datagen.flow(x_res, y_res, batch_size=128, subset='training')
        val_generator = train_datagen.flow(x_res, y_res, batch_size=128, subset='validation')

    return train_generator, val_generator

def run_experiment(learning_rates, optimizers, scenarios, epochs=20):
    results = []
    for scenario in scenarios:
        train_dataset, val_dataset = get_datasets(scenario)
        for lr in learning_rates:
            for optimizer_class in optimizers:
                with wandb.init(project="cnn-proyecto-1-ml", config={
                    "learning_rate": lr,
                    "epochs": epochs,
                    "optimizer": optimizer_class.__name__,
                    "scenario": scenario
                }) as run:
                    model = build_model()
                    optimizer = optimizer_class(learning_rate=lr)
                    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

                    for epoch in range(epochs):
                        history = model.fit(train_dataset, validation_data=val_dataset, epochs=1, verbose=1)
                        wandb.log({
                            'epoch': epoch + 1,
                            'train_loss': history.history['loss'][0],
                            'train_accuracy': history.history['accuracy'][0],
                            'val_loss': history.history['val_loss'][0],
                            'val_accuracy': history.history['val_accuracy'][0]
                        })

                    # Log final epoch metrics
                    results.append({
                        'optimizer': optimizer_class.__name__,
                        'learning_rate': lr,
                        'accuracy': history.history['val_accuracy'][0],
                        'loss': history.history['val_loss'][0]
                    })

    return results


In [ ]:
# Configuración de experimentos
learning_rates = [0.001, 0.0001, 0.00001]
scenarios = ['original', 'undersampling', 'oversampling']
optimizers = [tf.keras.optimizers.Adam, tf.keras.optimizers.SGD, tf.keras.optimizers.RMSprop]

# Ejecutar experimentos
experiment_results = run_experiment(learning_rates, optimizers, scenarios)

Found 16808 images belonging to 3 classes.
Found 4201 images belonging to 3 classes.


132/132 ━━━━━━━━━━━━━━━━━━━━ 60s 401ms/step - accuracy: 0.5577 - loss: 1.7785 - val_accuracy: 0.5561 - val_loss: 0.9381
132/132 ━━━━━━━━━━━━━━━━━━━━ 54s 393ms/step - accuracy: 0.7742 - loss: 0.5266 - val_accuracy: 0.5999 - val_loss: 1.9461
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step - accuracy: 0.8302 - loss: 0.3936